# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

## **0.미션**

단계 3에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [3]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [4]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

# 더 필요한 라이브러리 추가 -------------




In [ ]:
# def get_distance():
#   emergency = pd.read_csv(path + '응급실정보.csv')
#   emergency.head()

#   return 0

### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [5]:
emergency = pd.read_csv(path + '응급실정보.csv')
emergency.head()

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491


## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 임의의 두 좌표간 거리 계산
    * 응급실 데이터프레임을 열어서
    * 응급실 두 곳의 좌표를 확인하고
    * 두 지점의 거리를 계산해 봅시다
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [6]:
haversine((35.548238, 129.307011), (35.236029, 129.216492), unit='km')

35.672596270813266

#### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후
    * 가장 가까운 거리의 응급실 3 곳을 반환합니다.
* 이를 함수로 생성하고 테스트 해 봅시다.

In [7]:
def recommend_hospital1(위도, 경도):
    temp = emergency.copy()
    temp['거리'] = temp.apply(lambda x: haversine((위도, 경도), (x['위도'], x['경도']), unit='km'), axis=1)
    return temp.sort_values(by='거리').head(3)

In [8]:
recommend_hospital1(35.548238, 129.307011)

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011,0.000045
299,의료법인동강의료재단동강병원,울산광역시 중구 태화로 239 (태화동),지역응급의료센터,052-241-1114,052-241-1190,35.553410,129.301943,0.735461
252,울산병원,울산광역시 남구 월평로171번길 13 (신정동),지역응급의료센터,052-259-5000,052-259-5119,35.546217,129.323100,1.472821


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율 적입니다.
* 해결 방안 : 그래서 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정 합니다.
* hint :
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
    * 응급실 정보에서 해당 구간을 먼저 조회한 후
    * 거리 계산

In [9]:
def recommend_hospital2(위도, 경도):
    # 위도 경도에 ± 0.5 범위에서 먼저 조회
    temp = emergency.loc[emergency['위도'].between(위도-0.05, 위도+0.05) & emergency['경도'].between(경도-0.05, 경도+0.05)].copy()

    # 거리 계산
    temp['거리'] = temp.apply(lambda x: haversine((위도, 경도), (x['위도'], x['경도']), unit='km'), axis=1)
    return temp.sort_values(by='거리').head(3)

recommend_hospital2(35.548238, 129.307011)

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011,0.000045
299,의료법인동강의료재단동강병원,울산광역시 중구 태화로 239 (태화동),지역응급의료센터,052-241-1114,052-241-1190,35.553410,129.301943,0.735461
252,울산병원,울산광역시 남구 월평로171번길 13 (신정동),지역응급의료센터,052-259-5000,052-259-5119,35.546217,129.323100,1.472821


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [10]:
# def load_file(filepath):
#     with open(filepath, 'r') as file:
#         return file.readline().strip()

# map_key = load_file(path + 'map_key.txt')
# map_key = json.loads(map_key)
# c_id, c_key = map_key['c_id'], map_key['c_key']

def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

########################
# API 키 로드 및 환경변수 설정
db = load_file(path + 'map_key.json')

# json Decoding
db = json.loads(db)

#############
c_id, c_key = db['Client_ID'], db['Client_Secret']
#########################

In [11]:
#print(db)

#### 2) 함수 생성

In [12]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    # 'route'와 'trafast' 키가 존재하는지 확인하고 예외 처리
    try:
        dist = data['route']['trafast'][0]['summary']['distance']  # m(미터)
        dist = dist / 1000  # km로 변환
    except KeyError as e:
        print(f"응답 데이터에서 예상되는 키를 찾을 수 없음: {e}")
        return None

    # dist = response['route']['trafast'][0]['summary']['distance']  # m(미터)

    return dist

* 테스트

In [13]:
loc1 = (35.548238, 129.307011)
loc2 = (35.236029, 129.216492)

dist = get_dist(loc1[0], loc1[1], loc2[0], loc2[1], c_id, c_key)
dist

39.263

#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [14]:
def recommend_hospital3(start_lat, start_lng, c_id, c_key):
    # 위도 경도에 ± 0.5 범위에서 먼저 조회
    temp = emergency.loc[emergency['위도'].between(start_lat-0.05, start_lat+0.05) & emergency['경도'].between(start_lng-0.05, start_lng+0.05)].copy()
    # display(temp)

    # 거리 계산
    temp['거리'] = temp.apply(lambda x: get_dist(start_lat, start_lng, x['위도'], x['경도'], c_id, c_key), axis=1)
    return temp.sort_values(by='거리').head(3)

recommend_hospital3(35.548238, 129.307011, c_id, c_key)

응답 데이터에서 예상되는 키를 찾을 수 없음: 'route'


,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리
299,의료법인동강의료재단동강병원,울산광역시 중구 태화로 239 (태화동),지역응급의료센터,052-241-1114,052-241-1190,35.553410,129.301943,2.032
368,의료법인정안의료재단중앙병원,"울산광역시 남구 문수로 472, 중앙병원 (신정동)",지역응급의료센터,052-226-1100,052-226-1119,35.532088,129.304840,2.125
252,울산병원,울산광역시 남구 월평로171번길 13 (신정동),지역응급의료센터,052-259-5000,052-259-5119,35.546217,129.323100,2.330


## **Mission Complete!**

수고 많았습니다!